In [6]:
from backtester import Engine, Strategy
import pandas_ta as ta

In [9]:

class SMAStrategy(Strategy):
    def next(self):
        for ticker in self.tickers:
            if self.position_size(ticker) == 0:
                if self.indicator(ticker, 'sma12')[-1] > self.indicator(ticker,'sma24')[-1]:                
                    self.buy(ticker, 100)
            else:
                if self.indicator(ticker, 'sma12')[-1] < self.indicator(ticker, 'sma24')[-1]:
                    self.sell(ticker, 100)


def sma(ohlc,*params):
    lookback = params[0]
    return ta.sma(ohlc.Close, length=lookback)

In [10]:
import yfinance as yf
import pandas_ta as ta
amzn = yf.Ticker('AMZN').history(period='1y')

e = Engine(SMAStrategy)
e.add_ohlc('AMZN', amzn)
e.add_indicator('sma12', sma, 12)

e.add_indicator('sma24', sma, 24)
out = e.run()
out.keys()


100%|██████████| 252/252 [00:00<00:00, 4507.47it/s]


dict_keys(['portfolio', 'positions', 'trades', 'statistics'])

In [11]:
out['portfolio']

,AMZN,cash,total
2022-01-26,0.000000,10000.000000,10000.000000
2022-01-27,0.000000,10000.000000,10000.000000
2022-01-28,0.000000,10000.000000,10000.000000
2022-01-31,0.000000,10000.000000,10000.000000
2022-02-01,0.000000,10000.000000,10000.000000
...,...,...,...
2023-01-20,9725.000000,131.999969,9856.999969
2023-01-23,9751.999664,131.999969,9883.999634
2023-01-24,9631.999969,131.999969,9763.999939
2023-01-25,9718.000031,131.999969,9850.000000


In [14]:
out['statistics']

,value
return_ann,0.016197
volatility_ann,5.451470
sharpe_ratio,0.002971


In [34]:
from backtesting import Backtest, Strategy

from backtesting.test import SMA


class SmaCross(Strategy):
    n1 = 12 
    n2 = 24

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if not self.position:
            if self.sma1[-1] > self.sma2[-1]:
                self.buy(size=100)
        elif self.sma1[-1] < self.sma2[-1]:
            self.position.close()


bt = Backtest(amzn, SmaCross,
              cash=10000, commission=0,
              exclusive_orders=True)

output = bt.run()



In [37]:
output

Start                     2022-01-20 00:00:00
End                       2023-01-19 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    1.195219
Equity Final [$]                  9605.999756
Equity Peak [$]                       10000.0
Return [%]                          -3.940002
Buy & Hold Return [%]              -38.233304
Return (Ann.) [%]                   -3.955385
Volatility (Ann.) [%]                2.694977
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.940002
Avg. Drawdown [%]                   -3.940002
Max. Drawdown Duration        6 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -3.992706
Worst Trade [%]                     -3.992706
Avg. Trade [%]                    